<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/dev/custom_dataset_of_script_VAEGAN_voxelsize1_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [1]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
!wandb login
output.clear()
print("ok!")

ok!


In [3]:
#@title Configure dataset
#@markdown - Leave empty if you want to start a new run
#@markdown - Set `"run_id"` if you want to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
resume_id = "3op30gca" #@param {type:"string"}
#@markdown Enter project name (either `handtool-gan` or `tree-gan`)
project_name = "tree-gan" #@param ["tree-gan", "handtool-gan"]
#@markdown Enter dataset location.  
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - zipfile example: `/content/drive/My Drive/h/k/a.zip`
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
data_location = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip" #@param {type:"string"}
#@markdown - choose rotation augmentation on-the-fly 
#@markdown (augmentation only support file folder in data_location)
data_augmentation = True    #@param {type:"boolean"}
aug_rotation_type = "random rotation"  #@param ["random rotation", "axis rotation"]
#@markdown - specify the rotation axis [x,y,z] (only for aug_rotation_type = "axis rotation")
rotation_axis_x = 0    #@param {type:"number"}
rotation_axis_y = 1    #@param {type:"number"}
rotation_axis_z = 0    #@param {type:"number"}

#@markdown - resolution of the voxelized array (shape resolution**3)
resolution = "32"    #@param [32, 64]

#@markdown Model
#@markdown - select which model to train
selected_model = "VAEGAN"    #@param ["VAEGAN", "GAN", "VAE"]

#@markdown WANDB log
#@markdown - how many epochs before logging images/3D objects
log_interval = 10    #@param {type:"integer"}
#@markdown - how many samples per log
log_num_samples = 1    #@param {type:"integer"}


#adjust parameter datatype
resolution = int(resolution)

colab_config = {
    "aug_rotation_type": aug_rotation_type,
    "data_augmentation": data_augmentation,
    "rotation_axis": (rotation_axis_x,rotation_axis_y,rotation_axis_z),
    "data_location": data_location,
    "resume_id": resume_id,
    "selected_model": selected_model,
    "log_interval": log_interval,
    "log_num_samples": log_num_samples,
    "project_name": project_name,
    "resolution": resolution
}

for k, v in colab_config.items():
    print(f"=> {k:20}: {v}")


=> aug_rotation_type   : random rotation
=> data_augmentation   : True
=> rotation_axis       : (0, 1, 0)
=> data_location       : /content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip
=> resume_id           : 3op30gca
=> selected_model      : VAEGAN
=> log_interval        : 10
=> log_num_samples     : 1
=> project_name        : tree-gan
=> resolution          : 32


# To just train a model, no edits should be required in any cells below.

In [4]:
import os
from pathlib import Path
# os.environ["WANDB_MODE"] = "dryrun"

%cd /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/
if project_name == "tree-gan":
    %cd colab-treegan/
else:
    %cd colab-handtool/

dataset_path = Path(data_location)
run_path = "./"

!apt-get update

%pip install pytorch-lightning
%pip install trimesh
!apt install -y xvfb
%pip install trimesh xvfbwrapper
output.clear()
print('ok!')

ok!


In [5]:
import io
from io import BytesIO
import sys
import zipfile
import trimesh
import numpy as np
from argparse import Namespace, ArgumentParser

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from xvfbwrapper import Xvfb

In [6]:
entity="bugan"
if not resume_id:
    run_id = wandb.util.generate_id()
    resume = False
else:
    run_id = resume_id
    resume = True

run = wandb.init(project=project_name, id=run_id, entity=entity, resume=True, dir=run_path, group=selected_model)

print("run id: " + str(wandb.run.id))
print("run name: " + str(wandb.run.name))
wandb.watch_called = False

wandb: Currently logged in as: ryan-au (use `wandb login --relogin` to force relogin)


run id: 3op30gca
run name: comfy-haze-645


In [7]:
if not resume:
    #keep track of hyperparams
    config = Namespace(**colab_config)

    config.seed = 1234
    config.epochs = 3000
    #datamodule config
    config.batch_size = 32
else:
    #all config will be replaced by the stored one in wandb
    api = wandb.Api()
    previous_run = api.run(f"{entity}/{project_name}/{run_id}")
    config = Namespace(**previous_run.config)
        #selected_model may be not in config
    if hasattr(config,"selected_model"):
        selected_model = config.selected_model



#dataset

In [8]:
### load our package

#directly install using pip
if hasattr(config,"rev_number"):
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git@{rev_number}#egg=bugan
else:
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git#egg=bugan

import bugan
#EXTRACT package version
    #switch stdout to temperary stringIO
old_stdout = sys.stdout
temp_stdout = io.StringIO()
sys.stdout = temp_stdout
    #get version
%pip freeze | grep bugan
version = temp_stdout.getvalue()
rev_number = version.split("+g")[1].rstrip()
    #switch back stdout
sys.stdout = old_stdout
output.clear()

print("bugan package version: ",version)
print("bugan package revision number: ",rev_number)
config.rev_number = rev_number


from bugan.functionsPL import *
from bugan.modelsPL import *


run.tags.append(selected_model)

###     load dataset
np.random.seed(config.seed)

dataset_path = Path(config.data_location)
if config.data_location.endswith(".zip"):
    config.dataset = dataset_path.stem
else:
    config.dataset = "dataset_array_custom"

dataModule = DataModule_process(config, run, dataset_path)
config.num_data = dataModule.size

print("dataset name: ",config.dataset)
print("dataset path: ",dataset_path)

bugan package version:  bugan==0.1.1.dev21+gb133bf8

bugan package revision number:  b133bf8
dataset name:  tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000
dataset path:  /content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip


#train

In [ ]:
#set seed
torch.manual_seed(config.seed)
torch.autograd.set_detect_anomaly(True)

#render setup
vdisplay = Xvfb()
vdisplay.start()

#wandb logger setup
wandb_logger = WandbLogger(experiment=run, log_model=True)
#log config
wandb.config.update(config)
wandb.jupyter.Notebook(Namespace(save_code=True)).save_history()
wandb.save(os.path.join(wandb.run.dir, "code", "_session_history.ipynb"), base_path=wandb.run.dir)

checkpoint_path = str(Path(wandb.run.dir).absolute()/'checkpoint.ckpt')
callbacks = [SaveWandbCallback(config.log_interval, checkpoint_path)]


#model
if selected_model == "VAEGAN":
    MODEL_CLASS = VAEGAN
elif selected_model == "GAN":
    MODEL_CLASS = GAN
else:
    MODEL_CLASS = VAE_train

if resume:
    # Download file from the wandb cloud.
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
    extra_trainer_args = {'resume_from_checkpoint': checkpoint_path}
    model = MODEL_CLASS.load_from_checkpoint(checkpoint_path)
else:
    extra_trainer_args = {}
    model = MODEL_CLASS(**vars(config))

trainer = pl.Trainer(gpus=1, max_epochs = config.epochs, logger=wandb_logger, callbacks=callbacks,\
                     default_root_dir=wandb.run.dir, checkpoint_callback = None, **extra_trainer_args)


#train
trainer.fit(model, dataModule)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Unzipping /content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip to /tmp


Meshes: 100%|██████████| 8710/8710 [00:24<00:00, 351.28it/s]


Processed dataset size: 8710
Number of failed file: 0



  | Name          | Type          | Params
------------------------------------------------
0 | vae           | VAE           | 58.3 M
1 | discriminator | Discriminator | 1.8 M 


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
